In [25]:
import networkx as nx
import pandas as pd
import numpy as np
import pulp
import itertools

df = pd.read_csv("dataset/finished_dataset.csv")

# Load the graph from the Pickle file
graph_from_routes = nx.read_gpickle("graph_from_routes_wd.pkl")


In [26]:
for  i, data in graph_from_routes.nodes(data =True):
    print(i, data)

42454691 {'depot': False}
42429657 {'depot': False}
42432082 {'depot': False}
42430333 {'depot': False}
42442514 {'depot': True}
42456578 {'depot': False}
42437368 {'depot': False}
42438862 {'depot': False}
42444916 {'depot': False}
42432847 {'depot': False}
42438155 {'depot': False}
42432963 {'depot': True}
1241742563 {'depot': True}
42440012 {'depot': False}
42454994 {'depot': False}
42431684 {'depot': False}
8482948064 {'depot': False}
42435422 {'depot': True}
42438781 {'depot': True}
42437222 {'depot': False}
42451987 {'depot': False}
42458342 {'depot': False}
42446977 {'depot': False}
42448254 {'depot': False}
1506445821 {'depot': False}
42429838 {'depot': False}
596776613 {'depot': False}
42432990 {'depot': False}
42434148 {'depot': False}
42430521 {'depot': False}
1773076513 {'depot': False}


In [27]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,location_pickup,location_dropoff,pickup_graph_node,dropoff_graph_node
0,0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,12,15,1241742563,42431684
1,1,42,id2129090,1,2016-03-14 14:05:39,2016-03-14 14:28:05,1,-73.975090,40.758766,-73.953201,40.765068,N,1346,2016-03-14,22,15,42446977,42431684
2,2,58,id0256505,1,2016-03-14 15:04:38,2016-03-14 15:16:13,1,-73.994484,40.745087,-73.998993,40.722710,N,695,2016-03-14,17,46,42435422,1773076513
3,3,103,id3502065,1,2016-03-15 21:03:44,2016-03-15 21:14:37,1,-73.974014,40.757153,-73.949738,40.777092,N,653,2016-03-15,22,23,42446977,42448254
4,4,374,id3817493,2,2016-03-14 14:57:56,2016-03-14 15:15:26,1,-73.952881,40.766468,-73.978630,40.761921,N,1050,2016-03-14,15,13,42431684,42440012


In [28]:
graph = graph_from_routes.copy()

In [29]:
node_info = {}
l = 50.000 / 3600#m/h
# Iterate over edges and add distances to the dictionary
for u, v, d in graph.edges(data=True):
    node_info[(u, v)] = [
        d.get('weight', None),
        d.get('weight', None),
        d.get('weight', None),
    ]

In [30]:
#for i, node in enumerate(graph.nodes()):
#    graph.nodes[node]['id'] = i

In [31]:
def distance_matrix_from_graph(G):
    """
    Create a distance matrix from a networkx graph.
    """
    num_nodes = len(G.nodes)
    distance_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    for u in range(num_nodes):
        for v in range(num_nodes):
            if u != v:
                distance_matrix[u][v] = nx.shortest_path_length(G, source=u, target=v, weight='weight')
    return distance_matrix



In [32]:
class Vehicle():
    def __init__(self, id, pcapacity=4,gcapacity = 10, 
                 discharge_rate = 1.5, 
                 start_depot = 0, 
                 end_depot = 0 ):
        self.id = id
        self.pcapacity_ = pcapacity
        self.gcapacity_ = gcapacity
        self.r_ = discharge_rate
        self.charge = 100
        self.start_depot = start_depot
        self.end_depot = end_depot
        self.requests_ ={}
    def update_requests(self, id, start, dest):
        self.requests_[id] = (start,dest)
    def update_requests(self, request):
        self.requests_[request.id] = (request.start,request.dest)

    def update_charge(self,distance, speed = 50):
        self.charge -=  distance/speed * self.r_


In [33]:
from enum import Enum
class Charger(Enum):
    SLOW = (1,0.6)
    MEDIUM = (1,0.8)
    FAST = (2,0.8)

class Depot():
    def __init__(self, node_id, type = Charger.SLOW, vcapacity = 50, vehicles_now = 25) -> None:
        self.id = node_id
        self.r, self.omega = type.value
        self.vcapacity_ = vcapacity
        self.vehicles_now = vehicles_now


In [34]:
class Request():
    def __init__(self, id, people, start, end) -> None:
        self.id = id
        self.people_ = people
        self.start = start
        self.end = end 


In [35]:
def convert_reqs_to_obj(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    reqs_obj = {}
    for node, item in requests.items():
        amount = item["amount"]
        temp_reqs = []
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
        reqs_obj[node] = temp_reqs
    
    return reqs_obj

In [36]:
def convert_reqs_to_list(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    temp_reqs = []
    for node, item in requests.items():
        amount = item["amount"]
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
    print(len(temp_reqs))
    return temp_reqs

In [37]:
def get_requests(dataframe, required_info = ["amount","internal_ids", "id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    requests = {}
    pick_up_nodes = np.unique(dataframe["pickup_graph_node"])
    
    for node in pick_up_nodes:
        temp_dict = dataframe[dataframe["pickup_graph_node"]==node].to_dict()
        temp_dict["amount"] = len(temp_dict["id"])
        temp_dict["internal_ids"] = list(temp_dict["id"].keys())
        requests[node] = {x:temp_dict[x] for x in required_info}
    reqs_obj = convert_reqs_to_obj(requests)
    reqs_list = convert_reqs_to_list(requests)
    return requests, reqs_obj, reqs_list

In [38]:
def initialize_depots(graph, vehicles_amount = [20]*5,type = [Charger.FAST]*5 ):
    depots = []
    idx = 0
    for node_id, data in graph.nodes(data = True):
        if data["depot"]:
            depots += [Depot(node_id, type = type[idx], vehicles_now = vehicles_amount[idx])]
            idx += 1
    return depots

In [39]:
def initialize_vehicles_naive(depots):
    vehicles = []
    # Calculate the total number of vehicles at the beginning
    n_vehicles = sum(d.vehicles_now for d in depots)
    vehicle_id_counter = 0  # Start assigning IDs from 1
    for i, d in enumerate(depots):
        start_depot_id = d.id
        end_depot_id =depots[(i + 1) % len(depots)].id  # Ensures start and end depots are different
        for _ in range(d.vehicles_now):
            vehicles.append(Vehicle(id=vehicle_id_counter,
                                    pcapacity=4,
                                    gcapacity=10,
                                    discharge_rate=1,
                                    start_depot=start_depot_id,
                                    end_depot=end_depot_id))
            vehicle_id_counter += 1  # Increment the ID counter
    return vehicles, n_vehicles


In [64]:
depots = initialize_depots(graph, vehicles_amount = [10]*5)
vehicles, _ = initialize_vehicles_naive(depots)

In [65]:
depot_ids = [d.id for d in depots]  # Generate the list of depot ids

# Replace values with NaN where dropoff_graph_node is in the list of depot ids
df.loc[df['dropoff_graph_node'].isin(depot_ids), 'dropoff_graph_node'] = np.nan
mode_value = df['dropoff_graph_node'].mode()[0]
df['dropoff_graph_node'].fillna(mode_value, inplace=True)
df.loc[df['pickup_graph_node'].isin(depot_ids), 'pickup_graph_node'] = np.nan
mode_value = df['pickup_graph_node'].mode()[0]
df['pickup_graph_node'].fillna(mode_value, inplace=True)

In [66]:
_, _, requests = get_requests(df[:10])
for r in requests:
    if r.people_ >4:
        requests.remove(r)
        continue
    
    for d in depots:
        if r.end == d.id or r.start == d.id:
            requests.remove(r)
            print("removed")
            continue
    print(r.id, r.start, r.end, r.people_)



10
id3817493 42431684.0 42440012.0 1
id0871223 42431684.0 42456578.0 1
id3860883 42431684.0 42432847.0 1
id2875421 42432847.0 42431684.0 1
id0256505 42432847.0 1773076513.0 1
id3570508 42432847.0 42446977.0 2
id3276198 42440012.0 42446977.0 1
id2129090 42446977.0 42431684.0 1
id3502065 42446977.0 42448254.0 1
id2969048 42454994.0 42429657.0 2


In [67]:
def make_double_sense(G):
    for u in G.nodes():
        for v in G.nodes():
            # Check if the edge (u, v) does not already exist and if u != v
            if u != v and G.has_edge(u, v):
                # Add the edge (u, v) and (v, u)
                G.add_edge(v, u)
    return G

#graph = make_double_sense(graph)

In [68]:
edges_to_remove = [(u, v) for u, v in graph.edges() if u == v]
graph.remove_edges_from(edges_to_remove)

In [69]:
def create_problem2(graph,depots, vehicles, requests,node_info=node_info, name="VehicleRoutingProblem", subsets=[]):
    problem = pulp.LpProblem(name, pulp.LpMinimize)
    #Routing variable
    V = {}
    for a in vehicles:
        for u in graph.nodes():
            for v in graph.nodes():
                if graph.has_edge(u, v):
                    V[a.id,u, v]= pulp.LpVariable(f"V^{a.id}_{u}_{v}", cat='Binary') 
    
    #Request Assigning variable
    x = {}
    for a in vehicles:
        for r in requests:
            x[a.id,r.id]= pulp.LpVariable(f"x_{a.id},{r.id}", cat='Binary') 

    s = {}
    for a in vehicles:
        for v in graph.nodes():
                s[a.id,v]= pulp.LpVariable(f"s_{a.id},{v}") 

    for a in vehicles:
            problem += pulp.lpSum(x[a.id,r.id]*r.people_ for r in requests) <= a.pcapacity_

    #(4.9)
    for r in requests:
        problem += pulp.lpSum(x[a.id,r.id] for a in vehicles) == 1
    
    #for a in vehicles:
    #    problem += pulp.lpSum(x[a.id,r.id] for r in requests) <= 2

    M = 20000000
    for a in vehicles:
        for i in graph.nodes():
                for j in graph.nodes():
                    if j != a.start_depot and graph.has_edge(i, j):
                        problem +=s[a.id,i] + node_info[(i,j)][1] - M*(1 - V[a.id, i, j]) <=s[a.id,j]


    y = {}
    N = len(requests)+200100
    for a in vehicles:
            y[a.id]= pulp.LpVariable(f"y_{a.id}", cat='Binary') 
            problem += N*y[a.id]>= (pulp.lpSum(x[a.id, r.id] for r in requests)) 

    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        no_end = list(graph.nodes()).copy()
        no_end.remove(a.end_depot)
        no_start = list(graph.nodes()).copy()
        no_start.remove(a.start_depot)
        
        for v in temp_list:
            problem += pulp.lpSum(V[a.id,u, v] for u in no_end if graph.has_edge(u, v)) \
                    - pulp.lpSum(V[a.id,v, w] for w in no_start if graph.has_edge(v, w)) == 0
        
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        no_end = list(graph.nodes()).copy()
        no_end.remove(a.end_depot)
        no_start = list(graph.nodes()).copy()
        no_start.remove(a.start_depot)
        if a.start_depot == a.end_depot:
            continue
        problem +=   (pulp.lpSum(V[a.id, v, a.start_depot] for v in no_start if graph.has_edge( v,a.start_depot)) ) == 0
        problem +=   (pulp.lpSum(V[a.id, a.end_depot,  v] for v in no_end if graph.has_edge( a.end_depot, v)) ) == 0
#       
        problem +=   (pulp.lpSum(V[a.id, v, a.end_depot] for v in no_start if graph.has_edge( v,a.end_depot)) ) ==y[a.id]
        #for r in requests:
        problem +=   (pulp.lpSum(V[a.id, a.start_depot,  v] for v in no_end if graph.has_edge( a.start_depot, v)) ) == y[a.id]

    
    for a in vehicles:
        for r in requests:
                problem += pulp.lpSum(V[a.id,  v,r.start] for v in graph.nodes() if graph.has_edge( v,r.start)) >= x[a.id, r.id]
                problem += pulp.lpSum(V[a.id,  v,r.end] for v in graph.nodes() if graph.has_edge( v,r.end)) >= x[a.id, r.id]
    #            #== x[a.id, r.id] )
    
    #problem += pulp.lpSum(y[a.id,r.start]  for a in vehicles for r in requests)
    problem += pulp.lpSum(-x[a.id,r.id] for a in vehicles for r in requests)
    #problem += pulp.lpSum(V[a.id,u, v]*node_info[(u,v)][1] for a in vehicles for v in graph.nodes() for u in graph.nodes() if graph.has_edge(u, v))

    return problem, V, x,y

In [70]:
problem, V, x, y= create_problem2(graph,depots, vehicles, requests)
print(problem.numVariables())

8300


In [71]:
problem.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/ca666fee4d2a420981f693d9bb0b9cfb-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/ca666fee4d2a420981f693d9bb0b9cfb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8765 COLUMNS
At line 59376 RHS
At line 68137 BOUNDS
At line 76438 ENDATA
Problem MODEL has 8760 rows, 8300 columns and 36610 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -10 - 0.05 seconds
Cgl0002I 400 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 50 strengthened rows, 0 substitutions
Cgl0004I processed model has 8460 rows, 7830 columns (6330 integer (6330 of which binary)) and 35250 elements
Cutoff incre

1

In [72]:
for a in vehicles:
    #if pulp.value(y[a.id])==1:
        print(y[a.id], pulp.value(y[a.id]))

y_0 0.0
y_1 0.0
y_2 0.0
y_3 0.0
y_4 0.0
y_5 0.0
y_6 0.0
y_7 0.0
y_8 0.0
y_9 0.0
y_10 0.0
y_11 0.0
y_12 1.0
y_13 0.0
y_14 0.0
y_15 0.0
y_16 0.0
y_17 0.0
y_18 0.0
y_19 0.0
y_20 1.0
y_21 0.0
y_22 0.0
y_23 0.0
y_24 0.0
y_25 0.0
y_26 0.0
y_27 0.0
y_28 0.0
y_29 1.0
y_30 0.0
y_31 0.0
y_32 0.0
y_33 0.0
y_34 0.0
y_35 0.0
y_36 0.0
y_37 0.0
y_38 0.0
y_39 0.0
y_40 1.0
y_41 0.0
y_42 0.0
y_43 0.0
y_44 0.0
y_45 0.0
y_46 0.0
y_47 0.0
y_48 0.0
y_49 0.0


In [73]:
objective_value = pulp.value(problem.objective)
objective_value

-10.0

In [74]:
for a in vehicles:
    for r in requests:
        if pulp.value(x[a.id,r.id]) == 1:
            print(f"Vehicle {a.id} assigned to {r.id}")

Vehicle 12 assigned to id0256505
Vehicle 12 assigned to id3570508
Vehicle 20 assigned to id3817493
Vehicle 20 assigned to id3860883
Vehicle 20 assigned to id2969048
Vehicle 29 assigned to id0871223
Vehicle 29 assigned to id2875421
Vehicle 29 assigned to id3276198
Vehicle 29 assigned to id2129090
Vehicle 40 assigned to id3502065


In [75]:
for a in vehicles:
    for u, v in graph.edges():
        if pulp.value(V[a.id,u, v]) == 1:
            print(f"Vehicle {a.id} visits {u}-{v}")
            break

Vehicle 12 visits 42429657-42456578
Vehicle 20 visits 42429657-42456578
Vehicle 29 visits 42429657-42456578
Vehicle 40 visits 42429657-42442514


, best possible -10 (22.62 seconds)
Cbc0010I After 7500 nodes, 499 on tree, 1e+50 best solution, best possible -10 (22.80 seconds)
Cbc0010I After 7600 nodes, 496 on tree, 1e+50 best solution, best possible -10 (23.09 seconds)
Cbc0010I After 7700 nodes, 503 on tree, 1e+50 best solution, best possible -10 (23.31 seconds)
Cbc0010I After 7800 nodes, 503 on tree, 1e+50 best solution, best possible -10 (23.54 seconds)
Cbc0010I After 7900 nodes, 503 on tree, 1e+50 best solution, best possible -10 (23.74 seconds)
Cbc0010I After 8000 nodes, 497 on tree, 1e+50 best solution, best possible -10 (23.96 seconds)
Cbc0010I After 8100 nodes, 495 on tree, 1e+50 best solution, best possible -10 (24.14 seconds)
Cbc0010I After 8200 nodes, 496 on tree, 1e+50 best solution, best possible -10 (24.41 seconds)
Cbc0010I After 8300 nodes, 498 on tree, 1e+50 best solution, best possible -10 (24.64 seconds)
Cbc0010I After 8400 nodes, 498 on tree, 1e+50 best solution, best possible -10 (24.88 seconds)
Cbc0010I After

In [52]:
def check_road(id, depot_id):
    raw_road = {}
    lrr = 0
    for u, v in graph.edges():
        if pulp.value(V[id,u, v]) == 1:
            raw_road[u] = v 
            lrr+=1
    n = depot_id
    final_road = {}
    print(lrr,raw_road)
    print( depot_id)
    for i in range(lrr):
        print(i+1,n)
        final_road[n] = raw_road[n]
        n = raw_road[n]

    print(len(final_road.keys()))

    print(final_road)
    return final_road
        

In [53]:
vehicles[2].start_depot

42432963

In [54]:
vehicles[11].end_depot

IndexError: list index out of range

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/ec52987a58f341c9bff258abe08296f1-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/b6/t14lv28n5xqdhgq7zcrqwfkm0000gn/T/ec52987a58f341c9bff258abe08296f1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1765 COLUMNS
At line 11904 RHS
At line 13665 BOUNDS
At line 15326 ENDATA
Problem MODEL has 1760 rows, 1660 columns and 7338 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -10 - 0.03 seconds
Cgl0002I 80 variables fixed
Cgl0004I processed model has 1690 rows, 1556 columns (1256 integer (1256 of which binary)) and 6946 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 100 integer variables out of 1256 

In [ ]:
node = 11
final_road = check_road(node, vehicles[node].start_depot)

9 {42429657: 42456578, 42456578: 42432847, 42432847: 1241742563, 42432963: 42431684, 42431684: 42438781, 42438781: 42446977, 42446977: 596776613, 596776613: 1773076513, 1773076513: 42429657}
42432963
1 42432963
2 42431684
3 42438781
4 42446977
5 596776613
6 1773076513
7 42429657
8 42456578
9 42432847
9
{42432963: 42431684, 42431684: 42438781, 42438781: 42446977, 42446977: 596776613, 596776613: 1773076513, 1773076513: 42429657, 42429657: 42456578, 42456578: 42432847, 42432847: 1241742563}


In [ ]:
raw_road = {}
lrr = 0
for u, v in graph.edges():
        if pulp.value(V[vehicles[node].id,u, v]) == 1:
            raw_road[u] = v 
            lrr+=1


In [ ]:
raw_road[42431684.0]

42438781

In [ ]:
for a in vehicles:
    for r in requests:
        if pulp.value(x[a.id,r.id]) == 1:
            print(f"Vehicle {a.id} assigned to {r.id}")

Vehicle 11 assigned to id2875421
Vehicle 11 assigned to id0256505
Vehicle 11 assigned to id2129090


In [ ]:
_, _, requests = get_requests(df[:3])
for r in requests:
    print(r.id, r.start, r.end, r.people_)

3
id2875421 42432847.0 42431684.0 1
id0256505 42432847.0 1773076513.0 1
id2129090 42446977.0 42431684.0 1


In [ ]:
42435716.0

In [ ]:
vehicles[4].start_depot

42447330